In [1]:
import requests
import time 
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

### 각 지점 Name, ID 얻기

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.khoa.go.kr/swtc/mobile.do')

In [4]:
node = driver.find_elements(By.XPATH, '//*[@id="gisSelect"]/option')

In [5]:
obsList = []
for i in node :
    obsList.append ( [i.text, i.get_attribute('value') ])
driver.close()

In [6]:
print('sample [54] : ',obsList[54][0], obsList[54][1])
print('len : ',len(obsList))

sample [54] :  마량항 SO_0705
len :  171


### 목표값 json 데이터 형태 확인

In [11]:
url = 'https://www.khoa.go.kr/swtc/getWeeklyData.do?stDate=20270322&obsPostId=DT_0059'
response = requests.get(url)
data = response.json()
data

{'getWeeklyData': {'weeklyData': [{'searchDate': '2027-03-22',
    'obsPostName': '백령도',
    'obsLon': 124.736081,
    'obsLat': 37.95565,
    'lvl1': '00:24/low/저/49',
    'lvl2': '06:00/high/고/341',
    'lvl3': '12:31/low/저/19',
    'lvl4': '18:26/high/고/369',
    'flgView': 'Y',
    'dateSun': '2027/03/22',
    'dateMoon': '2027/02/15',
    'moonState': '대조기',
    'moolNormal': '가슴사리',
    'mool7': '여섯매',
    'mool8': '일곱물',
    'temp': '/'},
   {'searchDate': '2027-03-23',
    'obsPostName': '백령도',
    'obsLon': 124.736081,
    'obsLat': 37.95565,
    'lvl1': '00:58/low/저/32',
    'lvl2': '06:41/high/고/361',
    'lvl3': '13:12/low/저/19',
    'lvl4': '18:58/high/고/359',
    'flgView': 'Y',
    'dateSun': '2027/03/23',
    'dateMoon': '2027/02/16',
    'moonState': '대조기',
    'moolNormal': '턱사리',
    'mool7': '일곱매',
    'mool8': '여덟물',
    'temp': '/'},
   {'searchDate': '2027-03-24',
    'obsPostName': '백령도',
    'obsLon': 124.736081,
    'obsLat': 37.95565,
    'lvl1': '01:29/low/저

In [12]:
for i in data :
    print(i)

getWeeklyData
weeklyDataDTO
weeklyData


In [13]:
data['weeklyData'][5]

{'searchDate': '2027-03-27',
 'obsPostName': '백령도',
 'obsLon': 124.736081,
 'obsLat': 37.95565,
 'lvl1': '02:57/low/저/33',
 'lvl2': '09:20/high/고/361',
 'lvl3': '15:43/low/저/108',
 'lvl4': '20:58/high/고/288',
 'flgView': 'Y',
 'dateSun': '2027/03/27',
 'dateMoon': '2027/02/20',
 'moolNormal': '허리사리',
 'mool7': '한꺽기',
 'mool8': '열두물',
 'temp': '/'}

In [14]:
datetime.strptime('1:09:44 PM', '%I:%M:%S %p').strftime("%H:%M")
datetime.strptime('20250325', '%Y%m%d').strftime("%Y-%m-%d")

'2025-03-25'

In [26]:
def getData(obs, code):

    obsName = obs[0]
    obsId = obs[1]
    
    start_date = datetime(2026, 1, 1)
    end_date = datetime(2027, 12, 31)
    
    current_date = start_date
    while current_date <= end_date:
        try :
            dateString = current_date.strftime("%Y%m%d")
            ## 조석 json 데이터 url
            url_tide = f'https://www.khoa.go.kr/swtc/getWeeklyData.do?stDate={dateString}&obsPostId={obsId}'
            response = requests.get(url_tide)
            full_json = response.json()
            result = []
            for data in full_json['weeklyData'] :
                today = data['dateSun']
                today_moon = data['dateMoon']
                mool8 = data['mool8']
                obsPostName = data['obsPostName']
                obsLon = data['obsLon']
                obsLat = data['obsLat']
                lv1 = data['lvl1']
                lv2 = data['lvl2']
                lv3 = data['lvl3']
                lv4 = data['lvl4']
    
                today_dash = datetime.strptime(dateString, '%Y%m%d').strftime("%Y-%m-%d")
                ## 조석지점 위경도의 일출/일몰 데이터 url
                url_sun = f"https://api.sunrise-sunset.org/json?lat={obsLat}&lng={obsLon}&date={today_dash}&tzid=Asia/Seoul"
                sunData = requests.get(url_sun).json()
                sunrise = sunData['results']['sunrise']
                sunset =  sunData['results']['sunset']
                sunrise = datetime.strptime(sunrise, '%I:%M:%S %p').strftime("%H:%M")
                sunset = datetime.strptime(sunset, '%I:%M:%S %p').strftime("%H:%M")
                result.append(f'{obsName},{obsId},{today},{today_moon},{mool8},{obsLon},{obsLat},{lv1},{lv2},{lv3},{lv4},{sunrise},{sunset}')
                
            current_date += timedelta(days=7)

            lock.acquire()
            results.append(result)
            if today[8]=='0' :
                print(f'{code}/{len(obsList)} {obsName},{obsId},{today},{today_moon},{mool8},{obsLon},{obsLat},{lv1},{sunrise},{sunset}')
            lock.release()
    
        except TypeError : 
            print('***',result)   
        except requests.exceptions.Timeout:
            print(code,'timeout!! ret!')
            #break
            continue


In [ ]:
import threading

for n in range(0, len(obsList), 10) :
    lock = threading.Lock()
    threads = []
    results = []
    for i in range(n, n+10 ): 
        if i >= len(obsList) : break
        t = threading.Thread(target=getData, args=(obsList[i], i))
        t.start()
        threads.append(t)
        time.sleep(3)
        print('attach',i)

    print('wait... start round / n:',n)    
    for t in threads:
        t.join()

    print( f" write results\n")
    file = open('.\\TIDE_INFO.txt', 'a')
    for rst in results :
        for line in rst :
            try :
                file.write( f"{line}\n")
            except UnicodeEncodeError : 
                println(line, "************")
    print( f" end write results\n")
    file.close()

attach 0
attach 1
0/171 가거도,SO_0577,2026/01/07,2025/11/19,열한물,125.128889,34.050833,03:01/high/고/205,07:43,17:42
attach 2
1/171 가덕도,DT_0063,2026/01/07,2025/11/19,열한물,128.810933,35.024178,04:55/low/저/14,07:31,17:25
attach 3
2/171 강구항,SO_0737,2026/01/07,2025/11/19,열한물,129.391306,36.358583,04:45/high/고/18,07:32,17:19
attach 4
attach 5
3/171 강릉항,SO_0733,2026/01/07,2025/11/19,열한물,128.951333,37.772278,04:44/high/고/25,07:38,17:17
attach 6
4/171 강양항,SO_0581,2026/01/07,2025/11/19,열한물,129.344722,35.39,03:55/low/저/3,07:30,17:21
attach 7
5/171 강화대교,DT_0032,2026/01/07,2025/11/19,열한물,126.522222,37.731944,03:57/low/저/124,07:47,17:27
attach 8
6/171 강화외포,SO_0539,2026/01/07,2025/11/19,열한물,126.3725,37.700833,02:31/low/저/114,07:48,17:27
attach 9
wait... start round / n: 0
7/171 강화하리,SO_1257,2026/01/07,2025/11/19,열한물,126.286667,37.728056,02:40/low/저/98,07:48,17:27
8/171 거문도,DT_0031,2026/01/07,2025/11/19,열한물,127.308889,34.028333,06:07/low/저/25,07:34,17:33
9/171 거제도,DT_0029,2026/01/07,2025/11/19,열한물,128.69916